In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DATA_PATH = "/kaggle/input/jigsaw-multilingual-toxic-comment-classification/"
os.listdir(DATA_PATH)


In [ ]:
TEST_PATH = DATA_PATH + "test.csv"
VAL_PATH = DATA_PATH + "validation.csv"
TRAIN_PATH = DATA_PATH + "jigsaw-toxic-comment-train.csv"

val_data = pd.read_csv(VAL_PATH)
test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

In [ ]:
train_data.head()

In [ ]:
val_data.head()

In [ ]:
test_data.head()

# Wordcloud - For training, val and test data

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt 

def clean_comment(x):
    if type(x) == str:
        return x.replace("\n", " ")
    else:
        return ""

In [ ]:
text = train_data.apply(lambda row: clean_comment(row["comment_text"]), axis=1).to_string()
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1200, stopwords=set(STOPWORDS)).generate(text)
plt.figure(figsize = (7, 7), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

In [ ]:
text = val_data.apply(lambda row: clean_comment(row["comment_text"]), axis=1).to_string()
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1200, stopwords=set(STOPWORDS)).generate(text)
plt.figure(figsize = (7, 7), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

In [ ]:
text = test_data.apply(lambda row: clean_comment(row["content"]), axis=1).to_string()
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1200, stopwords=set(STOPWORDS)).generate(text)
plt.figure(figsize = (7, 7), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

## Findings 

1. Test and validation data doesn't seem to be really similar to training data. 
2. Training data shows a lot english words, but the validation and test data shows many non-english data. 
3. This is going to an issue. 

# About non-english comments

In [ ]:
!apt-get install -y python-numpy libicu-dev

!pip install pyicu==2.3.1

!pip install morfessor

!pip install pycld2


In [ ]:
from polyglot.detect import Detector
from polyglot.detect.langids import isoLangs

def detect_lang(comment):
    return Detector("".join(x for x in comment if x.isprintable()), quiet=True).languages[0].name
    
def get_lang_name(code):
    try:
        return isoLangs[code]["name"]
    except:
        return "Unknown" 

In [ ]:
train_data["lang_code"] = train_data.apply(lambda row: detect_lang(row["comment_text"]), axis=1)
train_data["lang_name"] = train_data["lang_code"].apply(lambda row: get_lang_name(row))

In [ ]:
print("\n------ Training data -------\n")
print("-> Unknown language code: ", train_data["lang_code"].loc[
    train_data["lang_name"] == "Unknown"].unique())
print("-> Unknown language examples count: ",train_data["lang_code"].loc[
    train_data["lang_name"] == "Unknown"].count() )
print("-> Unknown language value counts: \n", train_data["lang_code"].loc[
    train_data["lang_name"] == "Unknown"].value_counts())

In [ ]:
print("\n------ Validation data -------\n")
  
val_data["lang_name"] = val_data["lang"].apply(lambda row: get_lang_name(row))
print("-> Unknown language code: ", val_data["lang"].loc[
    val_data["lang_name"] == "Unknown"].unique())
print("-> Unknown language examples count: ",val_data["lang"].loc[
    val_data["lang_name"] == "Unknown"].count() )
print("-> Unknown language value counts: \n", val_data["lang"].loc[
    val_data["lang_name"] == "Unknown"].value_counts())

In [ ]:
print("\n------ Testing data -------\n")
  
test_data["lang_name"] = test_data["lang"].apply(lambda row: get_lang_name(row))
print("-> Unknown language code: ", test_data["lang"].loc[
    test_data["lang_name"] == "Unknown"].unique())
print("-> Unknown language examples count: ",test_data["lang"].loc[
    test_data["lang_name"] == "Unknown"].count() )
print("-> Unknown language value counts: \n", test_data["lang"].loc[
    test_data["lang_name"] == "Unknown"].value_counts())

## Findings 

1. There are 717 examples in the data whose language is unknown.
2. Among them, 616 are completely unknown (Language code: 'un')
3. Rest of them are not categorized by the polyglot package. ('jw', 'hmn', 'haw', 'nso', 'zu', 'zzp', 'tlh', 'kha', 'iw', 'mfe', 'crs', 'syr', 'xx')
3. Interestingly, testing and validation data has no such instances of an unknown language. 
4. Thus the unknown language comments can be removed from training data. 

# English Language v/s Non-English languages

In [ ]:
train_eng_count = train_data["lang_name"].loc[train_data["lang_name"] == "English"].count()
train_non_eng_count = train_data["lang_name"].loc[~train_data["lang_name"].isin(["English", "Unknown"])].count()

val_eng_count = val_data["lang_name"].loc[val_data["lang_name"] == "English"].count()
val_non_eng_count = val_data["lang_name"].loc[~val_data["lang_name"].isin(["English", "Unknown"])].count()

test_eng_count = test_data["lang_name"].loc[test_data["lang_name"] == "English"].count()
test_non_eng_count = test_data["lang_name"].loc[~test_data["lang_name"].isin(["English", "Unknown"])].count()


In [ ]:
x = ['English', 'Non-English']
lang_count = [[train_eng_count, train_non_eng_count], [val_eng_count, val_non_eng_count], 
          [test_eng_count, test_non_eng_count]]
lang_count_titles =  [ "Training data", "Validation data", "Testing data"]
x_pos = np.arange(len(x))

fig, ax = plt.subplots(1, 3, figsize=(15,5))

for i, col in enumerate(ax):
    tmp_data = lang_count[i]
    col.bar(x_pos, tmp_data)
#     plt.bar(x_pos, tmp_data)
    for j, v in enumerate(tmp_data):
        col.text(x_pos[j] - 0.1, v + .01, "{:.2f}%".format(v*100/sum(tmp_data)))
    col.set_title(lang_count_titles[i])
    col.set_xlabel("Language")
    col.set_ylabel("Count")
    col.set_xticks(x_pos)
    col.set_xticklabels(x)
    col.plot()


fig.suptitle("Language Count Across Datasets", fontsize = 33)
fig.tight_layout()
fig.subplots_adjust(top=0.8)

fig.show()

## Findings 

1. The data is a highly skewed. 99% of the comments are in English.
2. But the validation and test data contains only non-english comments. 
3. So viable solution is to translate these comments to english and build a model on it.
4. On validation and testing, first translate the comment to English and pass it to the model for prediction. 

# Major non-english languages

In [ ]:
train_non_eng_count = train_data["lang_name"][~train_data["lang_name"].isin(
    ['English', 'Unknown'])].value_counts().rename_axis('lang').reset_index(name='lang_count')

val_non_eng_count = val_data["lang_name"][~val_data["lang_name"].isin(
    ['English', 'Unknown'])].value_counts().rename_axis('lang').reset_index(name='lang_count')

test_non_eng_count = test_data["lang_name"][~test_data["lang_name"].isin(
    ['English', 'Unknown'])].value_counts().rename_axis('lang').reset_index(name='lang_count')

In [ ]:
df_set = [train_non_eng_count, val_non_eng_count, test_non_eng_count]
non_eng_count_titles =  [ "Training data", "Validation data", "Testing data"]

fig, ax = plt.subplots(1, 3, figsize=(18,5))

for i, col in enumerate(ax):
    tmp_data = df_set[i]
    
    if i == 0:
        mean_count = tmp_data.lang_count.mean()
        idx = tmp_data[tmp_data['lang_count'] > int(mean_count)].lang
        vals = tmp_data[tmp_data['lang_count'] > int(mean_count)].lang_count
    else:
        idx = tmp_data.lang
        vals = tmp_data.lang_count
    col.pie(vals, labels=idx, autopct='%1.1f%%')
    col.axis('equal')
    col.set_title(non_eng_count_titles[i])
    col.plot()

fig.suptitle("Non English Count Across Datasets", fontsize = 33)
fig.tight_layout()
fig.subplots_adjust(top=0.8)
plt.show()

## Findings

**Training data**
1. The top-5 non-english languages are German, Scot, Danish, Arabic and Spanish
2. They constitutes 34% of total contribution. 
3. Considered the languages with counts more than mean count, which is 17. 

**Validation Data**

1. Didn't adhere mean count criteria since there are only a few languages present in the data. 
2. There are only three languages present in validation dataset.
    - Turkish
    - Spanish
    - Italian
 
**Testing Data**

1. Didn't adhere mean count criteria since there are only a few languages present in the data. 
2. There are only six languages present in validation dataset.
    - Turkish
    - Spanish
    - Italian
    - French
    - Portugese
    - Russian

# Geographical distribution of comment data

English is a universal language. How other languages are distributed in the data geographically? 

In [ ]:
country_map = {
    'Unknown': 'Unknown',
    'English': 'United Kingdom',
    'Azerbaijani': 'Azerbaijan',
    'Basque': 'Spain',
    'Interlingua': 'Italy',
    'German': 'Germany',
    'Volapük': 'Germany',
    'Norwegian Nynorsk': 'Norway',
    'Scots': 'Scotland',
    'Dutch': 'Netherlands',
    'Polish': 'Poland',
    'Greek, Modern': 'Greece',
    'Manx': 'Ireland',
    'Portuguese': 'Portugal',
    'Luganda': 'Uganda',
    'Hungarian': 'Hungary',
    'Kinyarwanda': 'Rwanda',
    'Danish': 'Denmark',
    'Latin': 'Italy',
    'Western Frisian': 'Netherlands',
    'Galician': 'Spain',
    'Italian': 'Italy',
    'Fijian': 'Fiji',
    'Sanskrit (Saṁskṛta)': 'India',
    'Occitan': 'Spain',
    'Xhosa': 'South Africa',
    'Quechua': 'Peru',
    'Welsh': 'Wales',
    'Maltese': 'Malta',
    'Chichewa; Chewa; Nyanja': 'Zimbabwe',
    'Czech': 'Czech Republic',
    'Oromo': 'Ethiopia',
    'Tagalog': 'Philippines',
    'Romansh': 'Switzerland',
    'Turkish': 'Turkey',
    'Irish': 'Ireland',
    'Venda': '	South Africa',
    'Indonesian': 'Indonesia',
    'Icelandic': 'Iceland',
    'Uzbek': 'Uzbekistan',
    'Interlingue': 'Italy',
    'Malagasy': 'Madagascar',
    'Swedish': 'Sweden',
    'Breton': 'France',
    'Somali': 'Somalia',
    'Luxembourgish, Letzeburgesch': 'Luxembourg',
    'Swahili': 'Kenya',
    'Faroese': 'Denmark',
    'Wolof': 'Senegal',
    'Spanish; Castilian': 'Spain',
    'Tsonga': 'Mozambique',
    'Shona': 'Zambia',
    'French': 'France',
    'Tonga (Tonga Islands)': 'Tonga Islands',
    'Southern Sotho': 'South Africa',
    'Norwegian': 'Norway',
    'Lithuanian': 'Lithuania',
    'Malay': 'Malaysia',
    'Tamil': 'India',
    'Corsican': 'France',
    'Japanese': 'Japan',
    'Turkmen': 'Turkmenistan',
    'Scottish Gaelic; Gaelic': 'Scotland',
    'Nauru': 'Nauru',
    'Samoan': 'Samoa',
    'Estonian': 'Estonia',
    'Waray-Waray': 'Philippines',
    'Latvian': 'Latvia',
    'Albanian': 'Albania',
    'Slovak': 'Slovakia',
    'Haitian; Haitian Creole': 'Haiti',
    'Esperanto': 'Brazil',
    'Māori': 'New Zealand',
    'Bulgarian': 'Bulgaria',
    'Sundanese': 'Indonesia',
    'Finnish': 'Finland',
    'Tatar': 'Russia',
    'Afar': 'Ethiopia',
    'Romanian, Moldavian, Moldovan': 'Romania',
    'Chinese': 'China',
    'Tswana': 'South Africa',
    'Zhuang, Chuang': '',
    'Serbian': 'Serbia',
    'Cebuano': 'China',
    'Lingala': 'Democratic Republic of the Congo',
    'Catalan; Valencian': 'Spain',
    'Ukrainian': 'Ukraine',
    'Persian': 'Iran',
    'Marathi (Marāṭhī)': 'India',
    'Guaraní': 'Paraguay',
    'Korean': 'South Korea',
    'Arabic': 'UAE',
    'Bosnian': 'Bosnia',
    'Vietnamese': 'Vietnam',
    'Urdu': 'India',
    'Thai': 'Thailand',
    'Croatian': 'Croatia',
    'Bengali': 'India',
    'Kurdish': 'Iraq',
    'Malayalam': 'India',
    'Hindi': 'India',
    'Macedonian': 'Macedonia',
    'Aymara': 'Bolivia',
    'Afrikaans': 'Australia',
    'Georgian': 'Georgia',
    'Oriya': 'India',
    'Kannada': 'India',
    'Russian': 'Russia',
    'Tibetan Standard, Tibetan, Central': 'Tibet',
    'Gujarati': 'India',
    'Mongolian': 'Mangolia',
    'Khmer': 'Vietnam',
    'Kirundi': 'Tanzania',
    'Nepali': 'Nepal',
    'Sinhala, Sinhalese': '',
    'Burmese': 'Burma',
    'Kalaallisut, Greenlandic': '',
    'Panjabi, Punjabi': 'India',
    'Swati': 'South Africa',
    'Yoruba': 'Nigeria',
    'Kazakh': 'Kazakhstan',
    'Hausa': 'Nigeria',
    'Slovene': 'Slovenia',
    'Tigrinya': 'Ethiopia',
    'Pashto, Pushto': 'Pakistan',
    'Akan': 'Ghana',
    'Telugu': 'India',
    'Bislama': 'Republic of Vanuatu',
    'Igbo': 'Nigeria',
    'Belarusian': 'Belarus'
}

In [ ]:
train_data["country"] = train_data.apply(lambda row: country_map[row["lang_name"]], axis=1)
val_data["country"] = val_data.apply(lambda row: country_map[row["lang_name"]], axis=1)
test_data["country"] = test_data.apply(lambda row: country_map[row["lang_name"]], axis=1)

In [ ]:
import plotly.express as px

In [ ]:
country_data_train = train_data.country.value_counts().rename_axis('country').reset_index(name='count')
fig = px.choropleth(country_data_train.query("country != 'United Kingdom' and country != 'Unknown'"), locations="country", hover_name="country",
                     projection="natural earth", locationmode="country names", title="Training data - geographical distribution", color="count",
                     template="plotly", color_continuous_scale="agsunset")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.2
fig.show()

In [ ]:
country_data_val = val_data.country.value_counts().rename_axis('country').reset_index(name='count')
fig = px.choropleth(country_data_val.query("country != 'United Kingdom' and country != 'Unknown'"), locations="country", hover_name="country",
                     projection="natural earth", locationmode="country names", title="Validation data - geographical distribution", color="count",
                     template="plotly", color_continuous_scale="agsunset")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.2
fig.show()

In [ ]:
country_data_test = test_data.country.value_counts().rename_axis('country').reset_index(name='count')
fig = px.choropleth(country_data_test.query("country != 'United Kingdom' and country != 'Unknown'"), locations="country", hover_name="country",
                     projection="natural earth", locationmode="country names", title="Test data - geographical distribution", color="count",
                     template="plotly", color_continuous_scale="agsunset")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.2
fig.show()

## Findings 


1. The english language was not considered since it is a universal language.
2. That explains the absence or feeble presence of training data at USA, Canada etc. 
3. Also, the languages are allotted to majority speaking country. Eg. French is allotted only to France

**Training data**

1. According to the plot, comment data is coming from various countries all over the planet.
2. In Asia, India has a significant representation in the data.
3. Considering the population of China, surprisingly there is no instance of Mandarin. (But it could be in the list of unknown languages.)

**Validation data**

1. The validation data confines to only three countries
    - Turkey
    - Italy
    - Spain
    
**Test data**

1. The test data confines to only six countries
    - Turkey
    - Italy
    - Spain
    - Russia
    - Portugal
    - France

# Understanding categories of training & validation data

In [ ]:
import seaborn as sns

In [ ]:
train_data['label'] = 0
train_data.loc[(train_data.toxic == 1) | (train_data.obscene == 1) | 
           (train_data.insult == 1) | (train_data.identity_hate == 1) | 
           (train_data.severe_toxic == 1) | (train_data.threat == 1), 'label'] = 1

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,7), )
ax = sns.countplot(x="label", data=train_data)
for i, p in enumerate(ax.patches):
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2., height + 0.7,
        "{:.2f}%".format(train_data['label'].value_counts(normalize=True)[i]*100),
            ha="center", rotation=10)
ax.set_xticklabels(['Non-Toxic', 'Toxic'])
ax.set_xlabel("Comments")
ax.set_ylabel("Count")
ax.set_title('Toxic and Non-Toxic comments in training data', fontsize=14)
fig.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,7), )
ax = sns.countplot(x="toxic", data=val_data)
for i, p in enumerate(ax.patches):
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2., height + 0.7,
        "{:.2f}%".format(val_data['toxic'].value_counts(normalize=True)[i]*100),
            ha="center", rotation=10)
ax.set_xticklabels(['Non-Toxic', 'Toxic'])
ax.set_xlabel("Comments")
ax.set_ylabel("Count")
ax.set_title('Toxic and Non-Toxic comments in validation data', fontsize=14)
fig.show()

## Findings

1. Training data is heavily skewed. 90% of data is non-toxic. 
2. Validation data follows the same pattern. 

# Understanding categories with in toxic comments

In [ ]:
comment_cat_df = pd.melt(train_data[['toxic','severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']])

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,7), )
ax = sns.countplot(data=comment_cat_df[comment_cat_df.value == 1], 
                   x='variable', 
                  order=comment_cat_df[comment_cat_df.value == 1].variable.value_counts().index)
for i, p in enumerate(ax.patches):
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2., height + 0.7,
        "{:.2f}%".format(comment_cat_df[comment_cat_df.value==1].variable.value_counts(normalize=True)[i]*100),
            ha="center", rotation=10)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=10, ha="right")
    ax.set_xlabel("Toxic comments")
    ax.set_ylabel("Count")


ax.set_title('Categories within toxic comments', fontsize=14)
fig.show()

## Findings

1. Among the toxic comments, 43% is coming under `toxic` category. 
2. `obscene` and `toxic` categories overlaps the most.  
3. `threat` category messages are the least in the training set. 

# Save the updated datasets

In [ ]:
train_data.to_csv("notebook_one_train_data.csv", index = False)
val_data.to_csv("notebook_one_val_data.csv", index = False)
test_data.to_csv("notebook_one_test_data.csv", index = False)